# Build Docker Images and Register with ECR
This notebook is an extension of the build [CPU](https://github.com/daniel-fudge/sagemaker-tennis/blob/master/build-cpu.ipynb) and [GPU](https://github.com/daniel-fudge/sagemaker-tennis/blob/master/build-gpu.ipynb) notebooks in the related [Tennis](https://github.com/daniel-fudge/sagemaker-tennis) repo. If you are unfamiliar with building Docker images and registering them in AWS [ECR](https://aws.amazon.com/ecr/), please see the [Tennis](https://github.com/daniel-fudge/sagemaker-tennis) repo.

If working on your own fork, you may want to set the following.
```shell
git config --global user.name "Your Name"
git config --global user.email your.email@domain.com
```

#### ECR Access
When you first created an SakeMaker instance either you made or selected a custom role or the system created one for you.  I believe there has been a change in the service since I initially made my `AmazonSageMaker-ExecutionRole`.  To execute the push below I had to add the `AmazonEC2ContainerRegistryFullAccess` policy to my SageMaker [IAM](https://console.aws.amazon.com/iam) role.  Specifically the `ecr:InitiateLayerUpload` access had to be added for images other than the SageMaker images covered in the SageMaker full access policy.

#### Build and register the CPU container

In [18]:
!chmod -R 755 container

In [19]:
!./container/build_and_push.sh

Requesting CPU image
Login Succeeded
Login Succeeded
Sending build context to Docker daemon  3.113MB
Step 1/10 : ARG REGION=us-east-1
Step 2/10 : FROM 520713654638.dkr.ecr.$REGION.amazonaws.com/sagemaker-pytorch:1.1.0-cpu-py3
 ---> d374fb352c72
Step 3/10 : RUN pip install --upgrade pip
 ---> Using cache
 ---> b4524ab0cb43
Step 4/10 : COPY requirements.txt requirements.txt
 ---> Using cache
 ---> d914220df64a
Step 5/10 : RUN pip install -r requirements.txt
 ---> Using cache
 ---> 3964e488f753
Step 6/10 : ENV PATH="/opt/ml/code:${PATH}"
 ---> Using cache
 ---> 2ba02b18656e
Step 7/10 : COPY /src /opt/ml/code
 ---> f098ab19b6e5
Step 8/10 : RUN chmod -R 755 /opt/ml/code
 ---> Running in e4887c56c254
Removing intermediate container e4887c56c254
 ---> 88c7d553d274
Step 9/10 : ENV SAGEMAKER_SUBMIT_DIRECTORY /opt/ml/code
 ---> Running in 2c45897e93d5
Removing intermediate container 2c45897e93d5
 ---> 6e62ed907a84
Step 10/10 : ENV SAGEMAKER_PROGRAM train.py
 ---> Running in 1f796dd29fc4
Removing

#### Build and register the GPU container

In [20]:
!./container/build_and_push.sh gpu

Requesting GPU image
Login Succeeded
Login Succeeded
Sending build context to Docker daemon  3.113MB
Step 1/10 : ARG REGION=us-east-1
Step 2/10 : FROM 520713654638.dkr.ecr.$REGION.amazonaws.com/sagemaker-pytorch:1.1.0-gpu-py3
 ---> adbf113505a2
Step 3/10 : RUN pip install --upgrade pip
 ---> Using cache
 ---> 635764385479
Step 4/10 : COPY requirements.txt requirements.txt
 ---> Using cache
 ---> 9449a77875e9
Step 5/10 : RUN pip install -r requirements.txt
 ---> Using cache
 ---> 783e15fb6bc1
Step 6/10 : ENV PATH="/opt/ml/code:${PATH}"
 ---> Using cache
 ---> 537b2b3c690c
Step 7/10 : COPY /src /opt/ml/code
 ---> 5e6d4f64df69
Step 8/10 : RUN chmod -R 755 /opt/ml/code
 ---> Running in 0b83e4a15ed7
Removing intermediate container 0b83e4a15ed7
 ---> 4005e4c5db03
Step 9/10 : ENV SAGEMAKER_SUBMIT_DIRECTORY /opt/ml/code
 ---> Running in 220f6367bd88
Removing intermediate container 220f6367bd88
 ---> b41aa1fb2306
Step 10/10 : ENV SAGEMAKER_PROGRAM train.py
 ---> Running in 466007f40d4a
Removing

#### Setup Notebook for local execution

In [21]:
!/bin/bash ./utils/setup.sh

The user has root access.
SageMaker instance route table setup is ok. We are good to go.
SageMaker instance routing for Docker is ok. We are good to go!


#### Train the CPU model locally
Since we only want to check the functionality, we will start at day 2900.  There are only 2922 days worth of signals so we are only letting the model trade for 22 days.

In [ ]:
from sagemaker.estimator import Estimator
from sagemaker import get_execution_role

start_day = 2900
role = get_execution_role()
estimator = Estimator(role=role,
                      train_instance_count=1,
                      train_instance_type='local',
                      image_name='portfolio-optimization-cpu:latest',
                      hyperparameters={'start_day': start_day})
estimator.fit()

Creating tmpy256mg5d_algo-1-rtqam_1 ... 
Attaching to tmpy256mg5d_algo-1-rtqam_12mdone
algo-1-rtqam_1  | 2020-07-13 00:45:28,779 sagemaker-containers INFO     Imported framework sagemaker_pytorch_container.training
algo-1-rtqam_1  | 2020-07-13 00:45:28,783 sagemaker-containers INFO     No GPUs detected (normal if no gpus installed)
algo-1-rtqam_1  | 2020-07-13 00:45:28,797 sagemaker_pytorch_container.training INFO     Block until all host DNS lookups succeed.
algo-1-rtqam_1  | 2020-07-13 00:45:28,798 sagemaker_pytorch_container.training INFO     Invoking user training script.
algo-1-rtqam_1  | 2020-07-13 00:45:28,799 sagemaker-containers INFO     Module train does not provide a setup.py. 
algo-1-rtqam_1  | Generating setup.py
algo-1-rtqam_1  | 2020-07-13 00:45:28,800 sagemaker-containers INFO     Generating setup.cfg
algo-1-rtqam_1  | 2020-07-13 00:45:28,800 sagemaker-containers INFO     Generating MANIFEST.in
algo-1-rtqam_1  | 2020-07-13 00:45:28,800 sagemaker-containers INFO     Inst

### Train on SageMaker

#### Locate the ECR image just built and pushed

In [ ]:
import boto3

client = boto3.client('sts')
account = client.get_caller_identity()['Account']
region = boto3.Session().region_name
ecr_image = '{}.dkr.ecr.{}.amazonaws.com/portfolio-optimization-cpu:latest'.format(account, region)

print(ecr_image)

#### Submit the training job

In [ ]:
from sagemaker.estimator import Estimator
estimator = Estimator(role=role,
                      train_instance_count=1,
                      train_instance_type='ml.m5.large',
                      image_name=ecr_image,
                      hyperparameters={'start_day': start_day})
estimator.fit()

### Get the results

#### Get the bucket name

In [ ]:
from sagemaker.session import Session

sagemaker_session = Session()
bucket = sagemaker_session.default_bucket()
job_name = estimator._current_job_name
print(bucket)
print(job_name)

#### Copy and unpack the result archive
Since we didn't start trading until nearly the end, the results are as expected.

In [ ]:
import shutil

s3 = boto3.resource('s3')
key = '{}/output/output.tar.gz'.format(estimator._current_job_name)
print(key)
s3.Bucket(bucket).download_file(key, 'output.tar.gz')
shutil.unpack_archive('output.tar.gz')

In [ ]:
from IPython.display import Image
Image(filename='history.png') 

# References
- [Tennis Repo](https://github.com/daniel-fudge/sagemaker-tennis)
- [Amazon ECS](https://aws.amazon.com/ecs/)

#### SageMaker
- [SageMaker Example:  Extending PyTorch Container](https://github.com/awslabs/amazon-sagemaker-examples/tree/master/advanced_functionality/pytorch_extending_our_containers)
- [How Amazon SageMaker interacts with your Docker container for training](https://docs.aws.amazon.com/sagemaker/latest/dg/your-algorithms-training-algo.html)
- [SageMaker Python SDK](https://github.com/aws/sagemaker-python-sdk)
- [scikit-bring-your-own](https://github.com/awslabs/amazon-sagemaker-examples/blob/master/advanced_functionality/scikit_bring_your_own/scikit_bring_your_own.ipynb)
- [SageMaker PyTorch container](https://github.com/aws/sagemaker-pytorch-container)
- [SageMaker Instance types](https://aws.amazon.com/sagemaker/pricing/instance-types/)
- [SageMaker Instance prices](https://aws.amazon.com/sagemaker/pricing/)

#### Docker
- [Dockerfile](https://docs.docker.com/engine/reference/builder/)
- [Docker home page](http://www.docker.com)
- [Getting started with Docker](https://docs.docker.com/get-started/)
- [Dockerfile reference](https://docs.docker.com/engine/reference/builder/)
- [docker run reference](https://docs.docker.com/engine/reference/run/)